In [1]:
#import all libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
import xgboost
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import statistics
import shap
import math



In [2]:
#read in the csv
df_dating=pd.read_csv('Speed Dating Data.csv',encoding = "ISO-8859-1")
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
df_dating=df_dating.loc[:, 'iid':'amb3_s']
df_dating=df_dating.drop(['field','career' ,'from','idg','id','partner','undergra','dec','dec_o'],axis='columns') #dropping identifiers and string columns and decision variables

In [3]:
#make all the columns integers
df_dating=df_dating.fillna(-1)
df_dating.head()
df_dating['iid'].nunique()
df_dating.head()

,iid,gender,condtn,wave,round,position,positin1,order,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field_cd,mn_sat,tuition,race,imprace,imprelig,zipcode,income,goal,date,go_out,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s
0,1,0,1,1,10,7,-1.0,4,11.0,0,0.14,0,27.0,2.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,8.0,8.0,8.0,8.0,6.0,7.0,4.0,2.0,21.0,1.0,-1,-1,4.0,2.0,4.0,"60,521","69,487.00",2.0,7.0,1.0,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,9.0,7.0,7.0,6.0,5.0,7.0,6.0,2.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,1,0,1,1,10,7,-1.0,3,12.0,0,0.54,0,22.0,2.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,8.0,10.0,7.0,7.0,5.0,8.0,4.0,2.0,21.0,1.0,-1,-1,4.0,2.0,4.0,"60,521","69,487.00",2.0,7.0,1.0,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,7.0,8.0,7.0,8.0,5.0,6.0,7.0,5.0,1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,1,0,1,1,10,7,-1.0,10,13.0,1,0.16,1,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,21.0,1.0,-1,-1,4.0,2.0,4.0,"60,521","69,487.00",2.0,7.0,1.0,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,5.0,8.0,9.0,8.0,5.0,7.0,7.0,-1.0,1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,1,0,1,1,10,7,-1.0,5,14.0,1,0.61,0,23.0,2.0,30.0,5.0,15.0,40.0,5.0,5.0,7.0,8.0,9.0,8.0,9.0,8.0,7.0,7.0,2.0,21.0,1.0,-1,-1,4.0,2.0,4.0,"60,521","69,487.00",2.0,7.0,1.0,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,7.0,6.0,8.0,7.0,6.0,8.0,7.0,6.0,2.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,1,0,1,1,10,7,-1.0,7,15.0,1,0.21,0,24.0,3.0,30.0,10.0,20.0,10.0,10.0,20.0,8.0,7.0,9.0,6.0,9.0,7.0,8.0,6.0,2.0,21.0,1.0,-1,-1,4.0,2.0,4.0,"60,521","69,487.00",2.0,7.0,1.0,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,5.0,6.0,7.0,7.0,6.0,6.0,6.0,6.0,2.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [4]:
#trying to make all values ints
df_dating['zipcode'] = df_dating['zipcode'].replace(',','', regex=True)
df_dating['zipcode'] = df_dating['zipcode'].apply(pd.to_numeric)
# do the same for income
df_dating['income'] = df_dating['income'].replace(',','', regex=True)
df_dating['income'] = df_dating['income'].apply(pd.to_numeric)
# do the same for mn_sat
df_dating['mn_sat'] = df_dating['mn_sat'].replace(',','', regex=True)
df_dating['mn_sat'] = df_dating['mn_sat'].apply(pd.to_numeric)
# do the same for tuition
df_dating['tuition'] = df_dating['tuition'].replace(',','', regex=True)
df_dating['tuition'] = df_dating['tuition'].apply(pd.to_numeric)
#remove periods
df_dating["pid"] = df_dating["pid"].astype(int)

In [5]:
df_zip=df_dating[['iid','zipcode']]
part_zip_ls=[]
row=0
for pid in df_dating['pid']:
    try:
        part_zip=df_zip.loc[df_zip['iid']== pid,'zipcode'].unique()[0]
        part_zip_ls.append(part_zip)
        row=row+1
#         print('row:',row)
#         print(part_zip)
    except:
        part_zip_ls.append(-1)
df_dating['part_zip']=part_zip_ls


In [6]:
df_dating.head()

,iid,gender,condtn,wave,round,position,positin1,order,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field_cd,mn_sat,tuition,race,imprace,imprelig,zipcode,income,goal,date,go_out,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,part_zip
0,1,0,1,1,10,7,-1.0,4,11,0,0.14,0,27.0,2.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,8.0,8.0,8.0,8.0,6.0,7.0,4.0,2.0,21.0,1.0,-1.0,-1.0,4.0,2.0,4.0,60521,69487.0,2.0,7.0,1.0,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,9.0,7.0,7.0,6.0,5.0,7.0,6.0,2.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
1,1,0,1,1,10,7,-1.0,3,12,0,0.54,0,22.0,2.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,8.0,10.0,7.0,7.0,5.0,8.0,4.0,2.0,21.0,1.0,-1.0,-1.0,4.0,2.0,4.0,60521,69487.0,2.0,7.0,1.0,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,7.0,8.0,7.0,8.0,5.0,6.0,7.0,5.0,1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,14850
2,1,0,1,1,10,7,-1.0,10,13,1,0.16,1,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,21.0,1.0,-1.0,-1.0,4.0,2.0,4.0,60521,69487.0,2.0,7.0,1.0,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,5.0,8.0,9.0,8.0,5.0,7.0,7.0,-1.0,1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,92821
3,1,0,1,1,10,7,-1.0,5,14,1,0.61,0,23.0,2.0,30.0,5.0,15.0,40.0,5.0,5.0,7.0,8.0,9.0,8.0,9.0,8.0,7.0,7.0,2.0,21.0,1.0,-1.0,-1.0,4.0,2.0,4.0,60521,69487.0,2.0,7.0,1.0,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,7.0,6.0,8.0,7.0,6.0,8.0,7.0,6.0,2.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,45243
4,1,0,1,1,10,7,-1.0,7,15,1,0.21,0,24.0,3.0,30.0,10.0,20.0,10.0,10.0,20.0,8.0,7.0,9.0,6.0,9.0,7.0,8.0,6.0,2.0,21.0,1.0,-1.0,-1.0,4.0,2.0,4.0,60521,69487.0,2.0,7.0,1.0,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,5.0,6.0,7.0,7.0,6.0,6.0,6.0,6.0,2.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,33183


In [7]:
#create the zip_match
df_dating['zip_match']=(df_dating['zipcode']==df_dating['part_zip'])
df_dating['zip_match']=df_dating['zip_match'].astype(int)
df_dating.head()

,iid,gender,condtn,wave,round,position,positin1,order,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field_cd,mn_sat,tuition,race,imprace,imprelig,zipcode,income,goal,date,go_out,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,part_zip,zip_match
0,1,0,1,1,10,7,-1.0,4,11,0,0.14,0,27.0,2.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,8.0,8.0,8.0,8.0,6.0,7.0,4.0,2.0,21.0,1.0,-1.0,-1.0,4.0,2.0,4.0,60521,69487.0,2.0,7.0,1.0,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,9.0,7.0,7.0,6.0,5.0,7.0,6.0,2.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0
1,1,0,1,1,10,7,-1.0,3,12,0,0.54,0,22.0,2.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,8.0,10.0,7.0,7.0,5.0,8.0,4.0,2.0,21.0,1.0,-1.0,-1.0,4.0,2.0,4.0,60521,69487.0,2.0,7.0,1.0,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,7.0,8.0,7.0,8.0,5.0,6.0,7.0,5.0,1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,14850,0
2,1,0,1,1,10,7,-1.0,10,13,1,0.16,1,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,21.0,1.0,-1.0,-1.0,4.0,2.0,4.0,60521,69487.0,2.0,7.0,1.0,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,5.0,8.0,9.0,8.0,5.0,7.0,7.0,-1.0,1.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,92821,0
3,1,0,1,1,10,7,-1.0,5,14,1,0.61,0,23.0,2.0,30.0,5.0,15.0,40.0,5.0,5.0,7.0,8.0,9.0,8.0,9.0,8.0,7.0,7.0,2.0,21.0,1.0,-1.0,-1.0,4.0,2.0,4.0,60521,69487.0,2.0,7.0,1.0,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,7.0,6.0,8.0,7.0,6.0,8.0,7.0,6.0,2.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,45243,0
4,1,0,1,1,10,7,-1.0,7,15,1,0.21,0,24.0,3.0,30.0,10.0,20.0,10.0,10.0,20.0,8.0,7.0,9.0,6.0,9.0,7.0,8.0,6.0,2.0,21.0,1.0,-1.0,-1.0,4.0,2.0,4.0,60521,69487.0,2.0,7.0,1.0,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,5.0,6.0,7.0,7.0,6.0,6.0,6.0,6.0,2.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,33183,0


In [8]:
#split the data into X and y
X=df_dating.loc[:,df_dating.columns != 'match']
y=df_dating['match']

In [9]:
std_scl_ftrs =X.columns

num_trsfm = Pipeline(steps=[
    ('std_scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[('num', num_trsfm, std_scl_ftrs)])


In [10]:
#Split the data using group kfold or group shuffle splits within 3 loops.


best_mod_ls=[]
test_scores_ls=[]
test_sets_ls=[]
bl_score_ls=[]
for rand_state in range(5):
    groups=df_dating['wave']
    gss1 = GroupShuffleSplit(n_splits=3, train_size=.6, random_state=rand_state)
    for train_idx, other_idx in gss1.split(X, y, groups):
    #     print("TRAIN:", train_idx, "other:", other_idx)


        X_train = X.loc[train_idx]
        X_other = X.loc[other_idx].reset_index(drop=True)
        y_train = y.loc[train_idx]
        y_other = y.loc[other_idx].reset_index(drop=True)

        groups=X_other['zipcode']
        gss2 = GroupShuffleSplit(n_splits=1, train_size=.5, random_state=rand_state)
        for val_idx, test_idx in gss1.split(X_other, y_other, groups):
    #         print("VAL:", val_idx, "TEST:", test_idx)
            X_val=X_other.loc[val_idx].reset_index(drop=True)
            y_val=y_other.loc[val_idx].reset_index(drop=True)
            X_test=X_other.loc[test_idx].reset_index(drop=True)
            y_test=y_other.loc[test_idx].reset_index(drop=True)

    # X_train['wave'].unique()
    # X_val['wave'].unique()
    # X_test['wave'].unique()
    
    #create a prediction for baseline where we predict all class 1.
    #the baseline model will always predict match to maximize the true positive.
    #the recall will be 1 because we are always preicting positive.
    y_row=len(y_test)
    y_dummy_pred=[1] * y_row
    #calculate baseline F1 score
    bl_f1=f1_score(y_dummy_pred, y_test, average='macro')
    bl_score_ls.append(bl_f1)
    print(bl_f1)
    
    
    #after split we are going to preprocess
    #build x training data frame
    X_train_tran=preprocessor.fit_transform(X_train)
    feature_names=preprocessor.get_feature_names_out()
    df_train=pd.DataFrame(data=X_train_tran,columns=feature_names)
    y_val.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True) #changes series to a df
    
    #build x val dataframe
    X_val_tran=preprocessor.transform(X_val)
    feature_names=preprocessor.get_feature_names_out()
    df_val=pd.DataFrame(data=X_val_tran,columns=feature_names)
    y_val.reset_index(drop=True, inplace=True) #changes series to a df
    #print(df_test.shape,y_test.shape)
    
    
    #build x test dataframe
    X_test_tran=preprocessor.transform(X_test)
    feature_names=preprocessor.get_feature_names_out()
    df_test=pd.DataFrame(data=X_test_tran, columns=feature_names)
    y_test.reset_index(drop=True, inplace=True) #changes series to a df 
    models_ls=[]
    val_scores_ls=[]
    
    for l in np.logspace(-3,3,5):
        for alpha in np.logspace(-3,3,5):
            for g in np.linspace(0.1,0.4,5):
                param_grid = {"learning_rate": [0.03],
                          "n_estimators": [10000],
                          "random_state": [rand_state],
                          "missing": [np.nan], 
                          "colsample_bytree": [0.9],              
                          "subsample": [0.75], 
                          "use_label_encoder": [False]}
                XGB_clf = xgboost.XGBClassifier(reg_alpha=alpha, reg_lambda=l, gamma=g, n_jobs=-1)
                XGB_clf.set_params(**ParameterGrid(param_grid)[0]) 
                XGB_clf.fit(df_train, y_train, 
                        early_stopping_rounds=50, 
                        eval_set=[(df_val, y_val)], 
                        eval_metric='logloss',
                        verbose=False)
                y_val_pred = XGB_clf.predict(df_val, ntree_limit=XGB_clf.best_ntree_limit)
                val_scores_ls.append(f1_score(y_val, y_val_pred, average='macro'))
                models_ls.append(XGB_clf)
    #time to find the best model with the best val score
    best_val_score=np.argmax(val_scores_ls)
    best_model=models_ls[best_val_score]
    best_mod_ls.append(best_model)
    
    ## Calculate test score
    y_test_pred = best_model.predict(df_test)
    test_score = f1_score(y_test, y_test_pred, average='macro')
    test_scores_ls.append(test_score)  # Save test score
    
    test_set = X_test.reset_index(drop=True).copy()
    test_set['match'] = y_test
    test_set['pred_match'] = y_test_pred
    test_sets_ls.append(test_set)  # Save test set WITH predictions from best model of random state



0.1467436108821105
0.12282958199356911
0.14494117647058824
0.14765389582436506
0.14838300570703866


In [12]:
XGB_model_mean=statistics.mean(test_scores_ls)
XGB_model_std=np.std(test_scores_ls)
XGB_avg_bl=statistics.mean(bl_score_ls)


print(f'the overall mean is: {XGB_model_mean} and the standard deviation is: {XGB_model_std}')

the overall mean is: 0.6811867247888873 and the standard deviation is: 0.010924683820083785


In [13]:
#3 additional models:elastic net, support vector machine, KNN
def MLpipe_GroupShuffleSplit_F1(X,y,preproc, ML_alg,param):
    random_state=42
    best_models = []
    test_scores_ls = []
    random_state_ls=[1,2,3,4,5,6,7,8,9,10]
    bl_score_ls=[]
    for random_state in random_state_ls:
        model_name=str(clf)[:-2].lower()
        print(model_name)
        if param.get(f'{model_name}__random_state'):
            print(param)
            param[f'{model_name}__random_state']=[random_state]
        # first split to separate out the test set
        X_other, X_test, y_other, y_test = train_test_split(X,y,test_size = 0.2,random_state=random_state)
        print('y_test shape:',y_test.shape)
        gss = GroupShuffleSplit(n_splits=4, train_size=.75, random_state=random_state)
        #create bl score on test set
        
        y_row=len(y_test)
        y_dummy_pred=[1] * y_row
        #calculate baseline F1 score
        bl_f1=f1_score(y_dummy_pred, y_test, average='macro')
        bl_score_ls.append(bl_f1)
        print(bl_f1)
        
        #create the pipeline
        pipe=make_pipeline(preproc,ML_alg)
        # use GridSearchCV
        # GridSearchCV loops through all parameter combinations and collects the results 
        grid = GridSearchCV(pipe, param_grid=param, scoring = 'f1_macro' ,cv=gss, return_train_score = True, n_jobs=-1, verbose=True)
        print(X_other.shape,y_other.shape)
        grid.fit(X_other, y_other, groups=X_other['wave'])
        #save results to a df.
        results=pd.DataFrame(grid.cv_results_)
        print(results)    
        #save the model
        best_models.append(grid)
        #calc and save the test score
        y_test_pred= best_models[-1].predict(X_test)
        print('y_test_pred:', y_test_pred)
        test_score=f1_score(y_test, y_test_pred, average='macro')
        test_scores_ls.append(test_score)
        print('test score:',test_score)
    model_mean=statistics.mean(test_scores_ls)
    model_std=np.std(test_scores_ls)
    print(f'the overall mean of all 10 models is: {model_mean} and the standard deviation is: {model_std}')
    avg_bl=statistics.mean(bl_score_ls)
    return model_mean,model_std,test_scores_ls,best_models,avg_bl
        

In [14]:
#logistic regression
from sklearn.linear_model import LogisticRegression
grid={
     'logisticregression__penalty' : ['l1', 'l2'],
    'logisticregression__C' : np.logspace(-4, 4, 20),
    'logisticregression__solver' : ['liblinear']}
    
clf = LogisticRegression()

lr_model_mean,lr_model_std, lr_test_scores_ls,lr_best_models,lr_avg_bl=MLpipe_GroupShuffleSplit_F1(X,y,preprocessor, clf,grid)
print(lr_test_scores_ls)
print(lr_avg_bl)


logisticregression
y_test shape: (1676,)
0.14051282051282052
(6702, 111) (6702,)
Fitting 4 folds for each of 40 candidates, totalling 160 fits
    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.096784      0.005331         0.011791        0.001947   
1        0.105094      0.014803         0.007027        0.002007   
2        0.066179      0.014106         0.007844        0.002462   
3        0.065741      0.011081         0.009599        0.003013   
4        0.050995      0.013484         0.006594        0.002150   
5        0.070506      0.010287         0.005238        0.000696   
6        0.044453      0.006501         0.006039        0.000756   
7        0.081060      0.012108         0.007188        0.002604   
8        0.051392      0.003718         0.005766        0.000405   
9        0.097084      0.015580         0.005988        0.000709   
10       0.048278      0.004242         0.005271        0.000715   
11       0.112453      0.003491         0

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.022137      0.001224         0.004777        0.000570   
1        0.033693      0.001913         0.005598        0.002034   
2        0.030403      0.003337         0.005673        0.000376   
3        0.041932      0.004025         0.005018        0.000283   
4        0.027481      0.002894         0.005214        0.000020   
5        0.053609      0.002459         0.009016        0.003322   
6        0.033892      0.010144         0.006526        0.001759   
7        0.074485      0.019693         0.005871        0.000696   
8        0.044849      0.006440         0.006883        0.002804   
9        0.083781      0.009506         0.006819        0.002702   
10       0.040970      0.003589         0.005304        0.000350   
11       0.115057      0.007185         0.006603        0.001730   
12       0.057800      0.005055         0.006505        0.000636   
13       0.115657      0.009823         0.008227

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.026079      0.006282         0.004730        0.000366   
1        0.041021      0.004742         0.004982        0.000544   
2        0.025159      0.004632         0.010007        0.009430   
3        0.050849      0.009584         0.005100        0.000554   
4        0.034676      0.003805         0.008704        0.004726   
5        0.064441      0.008471         0.005505        0.000520   
6        0.030613      0.005513         0.006465        0.001770   
7        0.063739      0.008348         0.006991        0.003770   
8        0.049155      0.002773         0.007098        0.002525   
9        0.085502      0.014965         0.005654        0.001108   
10       0.042754      0.007429         0.005167        0.000393   
11       0.116036      0.014206         0.006073        0.001301   
12       0.063019      0.012744         0.006375        0.000853   
13       0.148588      0.026676         0.005906

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.027593      0.005351         0.005093        0.000341   
1        0.035659      0.002259         0.004836        0.000533   
2        0.027641      0.003407         0.005723        0.000284   
3        0.043738      0.005123         0.005195        0.000211   
4        0.034907      0.009442         0.005177        0.000144   
5        0.062277      0.005474         0.007180        0.002140   
6        0.030785      0.003261         0.005504        0.000532   
7        0.064156      0.005918         0.006454        0.001528   
8        0.045002      0.006148         0.005765        0.000684   
9        0.085567      0.004431         0.007614        0.003209   
10       0.044911      0.002134         0.006008        0.000453   
11       0.117292      0.014367         0.008669        0.002597   
12       0.050751      0.004656         0.006671        0.000829   
13       0.132359      0.015004         0.008359

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.021933      0.001227         0.004585        0.001054   
1        0.033073      0.002619         0.004559        0.000634   
2        0.025255      0.000747         0.004945        0.000405   
3        0.040617      0.005160         0.005672        0.001089   
4        0.027768      0.003228         0.005954        0.001637   
5        0.061583      0.014209         0.006887        0.002439   
6        0.027616      0.005172         0.007648        0.003702   
7        0.078133      0.024805         0.010030        0.007300   
8        0.045060      0.010180         0.010135        0.006616   
9        0.096730      0.026091         0.008004        0.004334   
10       0.041728      0.005280         0.010951        0.009482   
11       0.105417      0.020588         0.006678        0.001606   
12       0.046347      0.010228         0.007596        0.004072   
13       0.130049      0.024604         0.010141

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.025039      0.004187         0.005290        0.000813   
1        0.038718      0.006974         0.005308        0.001268   
2        0.024921      0.004036         0.004915        0.000514   
3        0.043866      0.008625         0.005714        0.001677   
4        0.028911      0.005358         0.005191        0.000509   
5        0.057179      0.015357         0.005326        0.000533   
6        0.032841      0.007662         0.006613        0.001636   
7        0.068035      0.022419         0.006777        0.001625   
8        0.043734      0.007066         0.005929        0.000571   
9        0.093216      0.017213         0.005540        0.001147   
10       0.046614      0.009015         0.009559        0.006764   
11       0.109379      0.018077         0.005521        0.001099   
12       0.047118      0.010854         0.005388        0.000657   
13       0.143608      0.038579         0.008268

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.021505      0.002302         0.005067        0.000386   
1        0.033446      0.004902         0.004774        0.000595   
2        0.024947      0.006515         0.005432        0.000115   
3        0.040179      0.004573         0.005037        0.000433   
4        0.023893      0.003087         0.005501        0.000311   
5        0.051376      0.003832         0.007565        0.002177   
6        0.035555      0.006670         0.005767        0.000580   
7        0.067174      0.012150         0.008509        0.005348   
8        0.044519      0.006526         0.006289        0.000390   
9        0.076131      0.020542         0.005492        0.000532   
10       0.034933      0.003912         0.006956        0.002918   
11       0.097040      0.019741         0.005925        0.000293   
12       0.043770      0.004588         0.007301        0.002120   
13       0.110218      0.006306         0.007249

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.023088      0.000930         0.004858        0.000475   
1        0.034324      0.002012         0.004353        0.000285   
2        0.027421      0.007094         0.004467        0.000346   
3        0.046611      0.003371         0.007311        0.003963   
4        0.034515      0.003089         0.009071        0.004689   
5        0.058247      0.006752         0.006865        0.003095   
6        0.031619      0.004674         0.007729        0.004165   
7        0.065459      0.004397         0.005383        0.000267   
8        0.053689      0.013356         0.005136        0.000283   
9        0.083568      0.010233         0.006725        0.002476   
10       0.042262      0.004754         0.006336        0.001656   
11       0.096470      0.003950         0.005413        0.000431   
12       0.048832      0.006320         0.005171        0.000233   
13       0.127670      0.012122         0.006537

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.022945      0.001645         0.004543        0.000277   
1        0.035704      0.005509         0.004025        0.000333   
2        0.027311      0.007178         0.004718        0.001031   
3        0.045113      0.004960         0.005840        0.001269   
4        0.030764      0.005680         0.004669        0.000602   
5        0.058351      0.011015         0.005625        0.000786   
6        0.030403      0.003715         0.006272        0.001854   
7        0.066370      0.008507         0.006842        0.003579   
8        0.045679      0.007905         0.005363        0.000536   
9        0.089213      0.020071         0.005132        0.000485   
10       0.040499      0.006840         0.005398        0.000654   
11       0.115598      0.027534         0.005416        0.000529   
12       0.055028      0.009991         0.005185        0.000313   
13       0.131803      0.032123         0.007178

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.022224      0.002639         0.005004        0.000408   
1        0.033587      0.004017         0.004732        0.000533   
2        0.023119      0.002085         0.004776        0.000518   
3        0.040482      0.006668         0.005759        0.000904   
4        0.028240      0.005144         0.005437        0.000743   
5        0.051552      0.004010         0.005616        0.000937   
6        0.036128      0.006117         0.007127        0.002611   
7        0.078767      0.023139         0.006698        0.002187   
8        0.042951      0.006988         0.005318        0.000717   
9        0.086108      0.017200         0.007748        0.004412   
10       0.041405      0.006303         0.005876        0.000498   
11       0.099234      0.009753         0.010232        0.005624   
12       0.050698      0.010852         0.006057        0.001744   
13       0.131297      0.020957         0.006539

In [15]:
#SVC
from sklearn.svm import SVC
# defining parameter range
grid = {'svc__C': [0.1, 1, 10, 100, 1000], 
              'svc__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'svc__kernel': ['rbf']} 
clf = SVC()
svc_model_mean,svc_model_std, svc_test_scores_ls,svc_best_models,svc_avg_bl=MLpipe_GroupShuffleSplit_F1(X,y,preprocessor, clf,grid)
print(svc_test_scores_ls)

  

svc
y_test shape: (1676,)
0.14051282051282052
(6702, 111) (6702,)
Fitting 4 folds for each of 25 candidates, totalling 100 fits
    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_svc__C  \
0        2.060392      0.344702         2.132261        0.248829          0.1   
1        1.991181      0.328792         2.061037        0.328245          0.1   
2        0.830060      0.072092         0.823850        0.137436          0.1   
3        0.675850      0.116789         0.700545        0.110471          0.1   
4        0.671413      0.121170         0.690332        0.116232          0.1   
5        1.926621      0.348719         2.084916        0.268425            1   
6        1.869890      0.352195         1.999552        0.289140            1   
7        0.779593      0.135795         0.774023        0.132522            1   
8        0.687584      0.131611         0.701298        0.114484            1   
9        0.664386      0.114570         0.678768        0.1096

y_test_pred: [0 0 0 ... 0 0 0]
test score: 0.702073973395754
svc
y_test shape: (1676,)
0.14489795918367346
(6702, 111) (6702,)
Fitting 4 folds for each of 25 candidates, totalling 100 fits
    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_svc__C  \
0        1.836665      0.201663       201.095113        0.111504          0.1   
1        1.792379      0.210126       201.070832        0.168284          0.1   
2        0.792019      0.122089         0.858568        0.058982          0.1   
3        0.637175      0.062724         0.719681        0.046066          0.1   
4        0.614109      0.071848         0.693134        0.037114          0.1   
5        1.778700      0.198222         2.195226        0.162188            1   
6        1.723340      0.216719         2.163521        0.115897            1   
7        0.743466      0.091835         0.836427        0.064601            1   
8        0.707058      0.125247         0.792410        0.076335            1   
9

y_test_pred: [0 0 0 ... 0 0 0]
test score: 0.7215936474593189
svc
y_test shape: (1676,)
0.14314928425357873
(6702, 111) (6702,)
Fitting 4 folds for each of 25 candidates, totalling 100 fits
    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_svc__C  \
0        2.316914      0.430834         1.876937        0.395000          0.1   
1        2.245791      0.378905         1.934816        0.349820          0.1   
2        1.002193      0.302450         0.736891        0.146678          0.1   
3        0.818224      0.171163         0.638314        0.130874          0.1   
4        0.763009      0.145158         0.599198        0.122276          0.1   
5        2.302042      0.465457         1.855465        0.405029            1   
6        2.269707      0.335507         1.813931        0.375346            1   
7        0.967373      0.261103         0.710583        0.140148            1   
8        0.805526      0.165688         0.605938        0.124498            1   


y_test_pred: [0 0 0 ... 0 0 0]
test score: 0.70671547890528
svc
y_test shape: (1676,)
0.14051282051282052
(6702, 111) (6702,)
Fitting 4 folds for each of 25 candidates, totalling 100 fits
    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_svc__C  \
0        1.967561      0.206201         2.027650        0.154619          0.1   
1        1.905831      0.179557         2.057783        0.175874          0.1   
2        0.837376      0.116903         0.835587        0.079834          0.1   
3        0.746024      0.078657         0.787149        0.068678          0.1   
4        0.754995      0.104844         0.794604        0.056436          0.1   
5        1.987675      0.244035         2.113349        0.145496            1   
6        1.813259      0.213970         2.085110        0.188275            1   
7        0.792469      0.094597         0.811043        0.051299            1   
8        0.732091      0.111526         0.765275        0.068165            1   
9 

y_test_pred: [0 0 0 ... 0 0 0]
test score: 0.7050038922978294
svc
y_test shape: (1676,)
0.13250517598343686
(6702, 111) (6702,)
Fitting 4 folds for each of 25 candidates, totalling 100 fits
    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_svc__C  \
0        1.855337      0.580344         1.985016        0.399882          0.1   
1        1.796609      0.555993         1.976165        0.390980          0.1   
2        0.812161      0.242033         0.830598        0.187769          0.1   
3        0.761217      0.224391         0.767281        0.178114          0.1   
4        0.733701      0.225963         0.730941        0.161456          0.1   
5        1.951085      0.606141         2.126931        0.396052            1   
6        1.833330      0.569134         2.153981        0.424899            1   
7        0.775304      0.239654         0.861006        0.204318            1   
8        0.715660      0.230509         0.710763        0.159190            1   


...

y_test_pred: [0 0 0 ... 0 0 0]
test score: 0.7122165796701576
svc
y_test shape: (1676,)
0.1391884951206985
(6702, 111) (6702,)
Fitting 4 folds for each of 25 candidates, totalling 100 fits
    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_svc__C  \
0        1.634659      0.318285         2.233079        0.220791          0.1   
1        1.552039      0.297597         2.186481        0.202400          0.1   
2        0.718175      0.163959         0.913247        0.110704          0.1   
3        0.607615      0.127127         0.784890        0.093205          0.1   
4        0.586297      0.132946         0.794129        0.068989          0.1   
5        1.612093      0.332090         2.165502        0.188287            1   
6        1.519530      0.323010         2.199281        0.204945            1   
7        0.676627      0.157389         0.945667        0.089823            1   
8        0.624321      0.162718         0.798052        0.075788            1   
9

y_test_pred: [0 0 1 ... 0 0 0]
test score: 0.7162785614643354
svc
y_test shape: (1676,)
0.15010141987829617
(6702, 111) (6702,)
Fitting 4 folds for each of 25 candidates, totalling 100 fits
    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_svc__C  \
0        1.849156      0.110957         2.124803        0.086567          0.1   
1        1.864381      0.114543         2.106762        0.119676          0.1   
2        0.782538      0.115768         0.804813        0.053103          0.1   
3        0.648460      0.053890         0.697212        0.031084          0.1   
4        0.631190      0.050348         0.679389        0.069935          0.1   
5        1.783612      0.123446         2.027368        0.120913            1   
6        1.765619      0.130281         2.047184        0.135008            1   
7        0.736640      0.077660         0.798618        0.045507            1   
8        0.653221      0.057829         0.674979        0.026117            1   


y_test_pred: [1 0 0 ... 0 0 0]
test score: 0.7053445850914206
svc
y_test shape: (1676,)
0.13519091847265222
(6702, 111) (6702,)
Fitting 4 folds for each of 25 candidates, totalling 100 fits
    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_svc__C  \
0        2.042806      0.427486         1.925920        0.366322          0.1   
1        2.002803      0.416889         1.946905        0.373079          0.1   
2        1.017221      0.187170         0.811878        0.191245          0.1   
3        0.732939      0.146229         0.695895        0.148929          0.1   
4        0.712903      0.128735         0.700994        0.178353          0.1   
5        2.218479      0.469272         1.979071        0.372003            1   
6        2.099680      0.459492         1.982682        0.369387            1   
7        0.825548      0.157533         0.780472        0.170266            1   
8        0.741633      0.148130         0.672561        0.156117            1   


y_test_pred: [0 0 0 ... 0 0 0]
test score: 0.7172773770407121
svc
y_test shape: (1676,)
0.13786008230452676
(6702, 111) (6702,)
Fitting 4 folds for each of 25 candidates, totalling 100 fits
    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_svc__C  \
0        1.885889      0.483661         2.044518        0.298035          0.1   
1        1.847390      0.468917         2.077540        0.311933          0.1   
2        0.802419      0.200309         0.838468        0.115850          0.1   
3        0.662376      0.166681         0.719613        0.116604          0.1   
4        0.644147      0.151944         0.701902        0.117740          0.1   
5        1.882397      0.475819         2.036377        0.281401            1   
6        1.724621      0.426350         1.995254        0.285881            1   
7        0.762250      0.175338         0.835366        0.132462            1   
8        0.660057      0.164949         0.734604        0.146769            1   


y_test_pred: [0 0 0 ... 0 0 0]
test score: 0.7016441614112208
the overall mean of all 10 models is: 0.7102062233680727 and the standard deviation is: 0.006608058470267999
[0.702073973395754, 0.7215936474593189, 0.70671547890528, 0.7050038922978294, 0.7139139769446983, 0.7122165796701576, 0.7162785614643354, 0.7053445850914206, 0.7172773770407121, 0.7016441614112208]


In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
grid={
    'kneighborsclassifier__leaf_size':[1,3,5,10,20,50],
    'kneighborsclassifier__n_neighbors':[1,3,5,10,20],
    'kneighborsclassifier__p':[1,2]
    
}
clf= KNeighborsClassifier()
knn_model_mean,knn_model_std, knn_test_scores_ls,knn_best_models,knn_avg_bl=MLpipe_GroupShuffleSplit_F1(X,y,preprocessor, clf,grid)
print(knn_test_scores_ls)


kneighborsclassifier
y_test shape: (1676,)
0.14051282051282052
(6702, 111) (6702,)
Fitting 4 folds for each of 60 candidates, totalling 240 fits


/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be elimina

/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be elimina

/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be elimina

/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be elimina

...

/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be elimina

/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be elimina

/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be elimina

/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be elimina

/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/henryjiang/opt/anaconda3/envs/data1030/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be elimina

In [ ]:
#comparison of all models mean and std f1 score with baseline 
model_ls=["XG_Boost","Logistic_Regression","SVC","KNN"]
avg_ls=[]
std_ls=[]
bl_ls=[]
avg_ls.extend([XGB_model_mean, lr_model_mean, svc_model_mean,knn_model_mean])
std_ls.extend([XGB_model_std, lr_model_std, svc_model_std,knn_model_std])
bl_ls.extend([XGB_avg_bl, lr_avg_bl, svc_avg_bl, knn_avg_bl])

plt.errorbar(x, y, e, linestyle='None', marker='^')
plt.plot(x, y, color='black',marker='o')

plt.show()

In [ ]:
#top global importance on the best model